https://www.datacamp.com/tutorial/llama3-fine-tuning-locally

In [1]:
# install all the necessary Python packages.

%pip install -U transformers 
%pip install -U datasets 
%pip install -U accelerate 
%pip install -U peft 
%pip install -U trl 
%pip install -U bitsandbytes 
%pip install -U wandb

     ---------------------------------------- 0.0/43.7 kB ? eta -:--:--
     ----------------- -------------------- 20.5/43.7 kB 330.3 kB/s eta 0:00:01
     -------------------------------------- 43.7/43.7 kB 428.4 kB/s eta 0:00:00
   ---------------------------------------- 0.0/9.5 MB ? eta -:--:--
   ---------------------------------------- 0.1/9.5 MB 2.0 MB/s eta 0:00:05
    --------------------------------------- 0.2/9.5 MB 2.9 MB/s eta 0:00:04
   - -------------------------------------- 0.5/9.5 MB 3.6 MB/s eta 0:00:03
   ---- ----------------------------------- 1.0/9.5 MB 5.5 MB/s eta 0:00:02
   ------ --------------------------------- 1.5/9.5 MB 7.4 MB/s eta 0:00:02
   --------- ------------------------------ 2.2/9.5 MB 9.2 MB/s eta 0:00:01
   ------------ --------------------------- 3.0/9.5 MB 9.4 MB/s eta 0:00:01
   --------------- ------------------------ 3.6/9.5 MB 11.3 MB/s eta 0:00:01
   --------------- ------------------------ 3.6/9.5 MB 11.3 MB/s eta 0:00:01
   ----------

In [3]:
# Import the necessary Python pages for loading the dataset, model, and tokenizer and fine-tuning.
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)

from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
import os, torch, wandb
from datasets import load_dataset
from trl import SFTTrainer, setup_chat_format

We’ll be tracking the training process using the Weights & Biases and then saving the fine-tuned model on Hugging Face, and for that, we have to log in to both Hugging Face Hub and Weights & Biases using the API key.

In [5]:
# The following code was updated using chatgpt so that it does not use kaggle secrets
from huggingface_hub import login
import json
import wandb

# Load the tokens from the configuration file
with open("config.json", "r") as file:
    config = json.load(file)

hf_token = config["HUGGINGFACE_TOKEN"]
wb_token = config["WANDB_TOKEN"]

# Login to Hugging Face
login(token=hf_token)

# Login to Weights & Biases
wandb.login(key=wb_token)
run = wandb.init(
    project='Fine-tune Llama 3 8B on Medical Dataset', 
    job_type="training", 
    anonymous="allow"
)


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Token is valid (permission: fineGrained).
Your token has been saved to C:\Users\USER\.cache\huggingface\token
Login successful


Set the base model, dataset, and new model variable. 
We’ll load the base model and the dataset from the HugginFace Hub and then save the new model.

In [6]:
# base_model = "/kaggle/input/llama-3/transformers/8b-chat-hf/1"
base_model = "meta-llama/Meta-Llama-3-8B"
dataset_name = "ruslanmv/ai-medical-chatbot"
new_model = "llama-3-8b-chat-doctor"

Set the data type and attention implementation.

In [7]:
torch_dtype = torch.float16
attn_implementation = "eager"

Check GPU readiness

In [8]:
import torch

print(torch.cuda.is_available())  # Should return True
print(torch.cuda.current_device())  # Should return the GPU device index, usually 0
print(torch.cuda.get_device_name(0))  # Should return the name of your GPU

True
0
NVIDIA GeForce RTX 3090


### Loading the model and tokenizer

In this part, we’ll load the model from huggingface. However, due to memory constraints, we’re unable to load the full model. Therefore, we’re loading the model using 4-bit precision.

Our goal in this project is to reduce memory usage and speed up the fine-tuning process.

In [9]:
# Check if GPU is available
if not torch.cuda.is_available():
    raise RuntimeError("No GPU found. A GPU is needed for quantization.")

# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=attn_implementation
)

Loading checkpoint shards: 100%|██████████| 4/4 [00:09<00:00,  2.30s/it]


Load the tokenizer and then set up a model and tokenizer for conversational AI tasks. By default, it uses the chatml template from OpenAI, which will convert the input text into a chat-like format.

In [10]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model)
model, tokenizer = setup_chat_format(model, tokenizer)

### Adding the adapter to the layer
Fine-tuning the full model will take a lot of time, so to improve the training time, we’ll attach the adapter layer with a few parameters, making the entire process faster and more memory-efficient.

In [11]:
# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)
model = get_peft_model(model, peft_config)

### Loading the dataset
To load and pre-process our dataset, we:

1. Load the ruslanmv/ai-medical-chatbot dataset, shuffle it, and select only the top 1000 rows. This will significantly reduce the training time.

2. Format the chat template to make it conversational. Combine the patient questions and doctor responses into a "text" column.

3. Display a sample from the text column (the “text” column has a chat-like format with special tokens).


#Importing the dataset

In [12]:
#Importing the dataset
dataset = load_dataset(dataset_name, split="all")
dataset = dataset.shuffle(seed=65).select(range(1000)) # Only use 1000 samples for quick demo

In [26]:
def format_chat_template(row):
    row_json = [{"role": "user", "content": row["Patient"]},
               {"role": "assistant", "content": row["Doctor"]}]
    row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row

dataset = dataset.map(
    format_chat_template,
    # num_proc=4,
)

dataset['text'][3]

Map: 100%|██████████| 1000/1000 [00:00<00:00, 17970.53 examples/s]


'<|im_start|>user\nFell on sidewalk face first about 8 hrs ago. Swollen, cut lip bruised and cut knee, and hurt pride initially. Now have muscle and shoulder pain, stiff jaw(think this is from the really swollen lip),pain in wrist, and headache. I assume this is all normal but are there specific things I should look for or will I just be in pain for a while given the hard fall?<|im_end|>\n<|im_start|>assistant\nHello and welcome to HCM,The injuries caused on various body parts have to be managed.The cut and swollen lip has to be managed by sterile dressing.The body pains, pain on injured site and jaw pain should be managed by pain killer and muscle relaxant.I suggest you to consult your primary healthcare provider for clinical assessment.In case there is evidence of infection in any of the injured sites, a course of antibiotics may have to be started to control the infection.Thanks and take careDr Shailja P Wahal<|im_end|>\n'

4. Split the dataset into a training and validation set.

In [13]:
dataset = dataset.train_test_split(test_size=0.1)

### Training the model
We are setting the model hyperparameters so that we can run it on the Kaggle. You can learn about each hyperparameter by reading the Fine-Tuning Llama 2 tutorial.

We are fine-tuning the model for one epoch and logging the metrics using the Weights and Biases.

In [14]:
training_arguments = TrainingArguments(
    output_dir=new_model,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=1,
    evaluation_strategy="steps",
    eval_steps=0.2,
    logging_steps=1,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    group_by_length=True,
    report_to="wandb"
)

c:\Users\USER\anaconda3\envs\myenv\Lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


We’ll now set up a supervised fine-tuning (SFT) trainer and provide a train and evaluation dataset, LoRA configuration, training argument, tokenizer, and model. We’re keeping the max_seq_length to 512 to avoid exceeding GPU memory during training.

In [15]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    peft_config=peft_config,
    max_seq_length=512,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

c:\Users\USER\anaconda3\envs\myenv\Lib\site-packages\huggingface_hub\utils\_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
c:\Users\USER\anaconda3\envs\myenv\Lib\site-packages\trl\trainer\sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
c:\Users\USER\anaconda3\envs\myenv\Lib\site-packages\trl\trainer\sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
Map:   0%|          | 0/900 [00:00<?, ? examples/s]


KeyError: 'text'